In [37]:
#import necessary libraries
import pandas as pd
import requests as req
from pandas import json_normalize
from matplotlib import pyplot as plt
import numpy as np
import openpyxl

In [2]:
#define function for calling match data using opendota API
#function accepts any number of match ids as input
def getMatchDetails(*matchids):
    #creating an empty dictionary to store the dataframes created for each api call
    match_dataframes = {}

    #for loop cycling through each match id, concatenating the id with url for the api call
    for matchid in matchids:
        url = f"https://api.opendota.com/api/matches/{matchid}"
        response = req.get(url)
    
        #if response code is good, store data as json, turn json data into dataframe, name the dataframe using the match id and store in the dictionary
        if response.status_code == 200:
            json_data = response.json()
            df = pd.DataFrame([json_data])
            match_dataframes[str(matchid)] = df
            print(f"DataFrame for Match ID {matchid} created successfully.")
        else:
            print(f"Failed to retrieve data for Match ID {matchid}. Status Code: {response.status_code}")

    #funtion returns the dictionary of dataframes
    return match_dataframes


In [3]:
#request hero data and make a dataframe from results called hero_data
url = "https://api.opendota.com/api/heroes"
get_heroes = req.get(url)
print(get_heroes.status_code)
get_heroes = get_heroes.json()
hero_data = pd.DataFrame(get_heroes)

200


In [4]:
#using the fuction created above, creating a dictionary of dataframes where each dataframe is the match data for each of the 151 games of the international 2023 tourney
international2023matches = getMatchDetails(7406531302,7406482053,7406424070,7406249246,7406129687,7404938247,7404889653,7404828649,7404763579,7404713057,7404668056,7404577536,7404488494,7404381764,7404249421,
                                           7403117150,7403057517,7402993316,7402943509,7402900929,7402800418,7402717428,7402626373,7402531427,7395420827,7395366323,7395312177,7395260041,7395201088,7395136494,
                                           7395048377,7394936686,7394832000,7393504676,7393464447,7393394521,7393331922,7393258586,7393133836,7393035065,7392908789,7391726311,7391681832,7391636050,7391585748,
                                           7391525205,7391463411,7391367511,7391247517,7391149823,7384188114,7384142800,7384108524,7384067209,7384021535,7383951137,7383855952,7383785611,7383689600,7382383379,
                                           7382316646,7382271393,7382234341,7382176499,7382126763,7382010427,7381903769,7381789226,7380503658,7380456695,7380454553,7380419098,7380411342,7380407733,7380401748,
                                           7380403482,7380377685,7380359415,7380353553,7380345561,7380339950,7380311197,7380292188,7380278774,7380279835,7380271573,7380204297,7380199457,7380191534,7380195136,
                                           7380184614,7380112773,7380101603,7380091953,7380089990,7379995208,7379995137,7379995224,7379995166,7379995104,7379050474,7379021143,7379016573,7379009384,7379001957,
                                           7378995240,7378986342,7378981947,7378973550,7378970387,7378962241,7378947046,7378947083,7378939289,7378932814,7378930433,7378917656,7378912368,7378905651,7378903471,
                                           7378901250,7378884672,7378880402,7378874061,7378876465,7378853458,7378851814,7378829327,7378828888,7378824942,7378805797,7378785114,7378776322,7378772656,7378760136,
                                           7378751246,7378707167,7378710682,7378693411,7378693331,7378676993,7378630668,7378617754,7378605383,7378609139,7378603271,7378530488,7378530439,7378530406,7378530507,
                                           7378530387
)

DataFrame for Match ID 7406531302 created successfully.
DataFrame for Match ID 7406482053 created successfully.
DataFrame for Match ID 7406424070 created successfully.
DataFrame for Match ID 7406249246 created successfully.
DataFrame for Match ID 7406129687 created successfully.
DataFrame for Match ID 7404938247 created successfully.
DataFrame for Match ID 7404889653 created successfully.
DataFrame for Match ID 7404828649 created successfully.
DataFrame for Match ID 7404763579 created successfully.
DataFrame for Match ID 7404713057 created successfully.
DataFrame for Match ID 7404668056 created successfully.
DataFrame for Match ID 7404577536 created successfully.
DataFrame for Match ID 7404488494 created successfully.
DataFrame for Match ID 7404381764 created successfully.
DataFrame for Match ID 7404249421 created successfully.
DataFrame for Match ID 7403117150 created successfully.
DataFrame for Match ID 7403057517 created successfully.
DataFrame for Match ID 7402993316 created succes

In [5]:
#defining a function that takes the dictionary of dataframes created above and makes a new singular dataframe containing all of the drafting phase data for all 151 matches
def mergedrafttimings(dataframes_dict):
    #creating an empty master dataframe
    master = pd.DataFrame()

    #looping through each dataframe in the dictionary
    for df_name, dataframe in dataframes_dict.items():
        #normalizing the data in the draft timings column
        drafttimings = pd.json_normalize(dataframe['draft_timings'])
        #creating a new dataframe of the normalized data
        drafttimingsmerged =pd.DataFrame(drafttimings.values.squeeze().tolist())
        #adding a 'match' column containing the match id
        drafttimingsmerged['match'] = df_name
        drafttimingsmerged['winning_team'] = dataframe['radiant_win']
        
        #concat the created dataframe to the master dataframe
        master = pd.concat([master, drafttimingsmerged], ignore_index=True)
    #returns the master dataframe
    return master


In [39]:
#executing the function created above and store the dataframe as draftdata
draftdata = mergedrafttimings(international2023matches)

In [42]:
#using the hero_data dataframe created earlier we are adding the heros actual name to our dataframe instead of having a numeric id
#merging the draft data frame with the hero_data frame using the id and hero_id as keys
draftdata = pd.merge(draftdata, hero_data[['id', 'localized_name']], left_on='hero_id', right_on='id', how='left')
#now that the hero name is added to the draftdata data frame we are dropping the hero_id and Id columns
draftdata.drop(['hero_id', 'id'], axis=1, inplace=True)

#renaming columns
draftdata.rename(columns={'localized_name': 'hero'}, inplace=True)
draftdata.rename(columns={'pick': 'action'}, inplace=True)

#remapping true and false values to say pick or ban
draftdata['action'] = draftdata['action'].map({True: 'pick', False: 'ban'})
#remapping team ids to say dire or radiant
draftdata['active_team'] = draftdata['active_team'].map({3: 'dire', 2: 'radiant'})
#remapping winning team field to say dire or radiant
draftdata['winning_team'] = draftdata['winning_team'].map({True: 'radiant', False: 'dire'})

#filling in all NaN values of winning team columns with respective true or false values
draftdata['winning_team'] = draftdata.groupby('match')['winning_team'].transform(lambda x: x.ffill().bfill())

#adding win column to show if the pick/ban resulted in a win or loss
draftdata['win'] = draftdata['winning_team'] == draftdata['active_team']  

#reordering columns
column_order = ['match','winning_team','order','action','active_team','hero','player_slot','extra_time','total_time_taken','win']
draftdata = draftdata[column_order]



In [43]:
draftdata

,match,winning_team,order,action,active_team,hero,player_slot,extra_time,total_time_taken,win
0,7406531302,dire,1,ban,dire,Chen,NaN,130,0,True
1,7406531302,dire,2,ban,radiant,Weaver,NaN,130,11,False
2,7406531302,dire,3,ban,radiant,Bristleback,NaN,130,10,False
3,7406531302,dire,4,ban,dire,Necrophos,NaN,130,45,True
4,7406531302,dire,5,ban,radiant,Magnus,NaN,130,17,False
...,...,...,...,...,...,...,...,...,...,...
3605,7378530387,radiant,20,ban,radiant,Invoker,NaN,80,29,True
3606,7378530387,radiant,21,ban,radiant,Monkey King,NaN,28,93,True
3607,7378530387,radiant,22,ban,dire,Tidehunter,NaN,15,42,False
3608,7378530387,radiant,23,pick,dire,Zeus,5.0,18,3,False


In [32]:
draftdata.to_excel('draft_data.xlsx', index=False)

for matplot lib i want to make a bar graph
3 bars for each hero:total times picked, total times banned, total wins

i need to create a list of all heros - done!
a list of corrosponding times picked - 
a list of corrosponding times banned
and a list of corrosponding wins


In [36]:
heroes_x = hero_data['localized_name'].to_list()

for hero in heroes_x:
    

['Anti-Mage',
 'Axe',
 'Bane',
 'Bloodseeker',
 'Crystal Maiden',
 'Drow Ranger',
 'Earthshaker',
 'Juggernaut',
 'Mirana',
 'Morphling',
 'Shadow Fiend',
 'Phantom Lancer',
 'Puck',
 'Pudge',
 'Razor',
 'Sand King',
 'Storm Spirit',
 'Sven',
 'Tiny',
 'Vengeful Spirit',
 'Windranger',
 'Zeus',
 'Kunkka',
 'Lina',
 'Lion',
 'Shadow Shaman',
 'Slardar',
 'Tidehunter',
 'Witch Doctor',
 'Lich',
 'Riki',
 'Enigma',
 'Tinker',
 'Sniper',
 'Necrophos',
 'Warlock',
 'Beastmaster',
 'Queen of Pain',
 'Venomancer',
 'Faceless Void',
 'Wraith King',
 'Death Prophet',
 'Phantom Assassin',
 'Pugna',
 'Templar Assassin',
 'Viper',
 'Luna',
 'Dragon Knight',
 'Dazzle',
 'Clockwerk',
 'Leshrac',
 "Nature's Prophet",
 'Lifestealer',
 'Dark Seer',
 'Clinkz',
 'Omniknight',
 'Enchantress',
 'Huskar',
 'Night Stalker',
 'Broodmother',
 'Bounty Hunter',
 'Weaver',
 'Jakiro',
 'Batrider',
 'Chen',
 'Spectre',
 'Ancient Apparition',
 'Doom',
 'Ursa',
 'Spirit Breaker',
 'Gyrocopter',
 'Alchemist',
 'Invoke